<a href="https://colab.research.google.com/github/ken-mao-hk/gltf/blob/main/boundingbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade ifcopenshell && pip install --upgrade numpy

In [ ]:
!pip install  --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape
import numpy as np


def get_bbox_vertices(bbox_min, bbox_max):
    """Generate all 8 vertices of a bounding box from min/max coordinates"""
    return [
        [bbox_min[0], bbox_min[1], bbox_min[2]],  # 0: min XYZ
        [bbox_max[0], bbox_min[1], bbox_min[2]],  # 1: max X
        [bbox_max[0], bbox_max[1], bbox_min[2]],  # 2: max XY
        [bbox_min[0], bbox_max[1], bbox_min[2]],  # 3: max Y
        [bbox_min[0], bbox_min[1], bbox_max[2]],  # 4: max Z
        [bbox_max[0], bbox_min[1], bbox_max[2]],  # 5: max XZ
        [bbox_max[0], bbox_max[1], bbox_max[2]],  # 6: max XYZ
        [bbox_min[0], bbox_max[1], bbox_max[2]]  # 7: max YZ
    ]


def local_to_global(transform_matrix, local_point):
    """使用形状几何自带的转换矩阵进行坐标转换"""
    local_homogeneous = np.array([*local_point, 1], dtype=np.float64)
    global_homogeneous = transform_matrix @ local_homogeneous
    global_point = global_homogeneous[:3] / global_homogeneous[3]  # 归一化处理
    return global_point.tolist()


def get_placement_matrix(placement):
    """从IfcLocalPlacement获取转换矩阵"""
    if placement.PlacementRelTo is None:
        parent_matrix = np.eye(4)
    else:
        parent_matrix = get_placement_matrix(placement.PlacementRelTo)

    axis2placement = placement.RelativePlacement
    location = np.array(axis2placement.Location.Coordinates)
    # 将毫米转换为米
    location = location / 1000.0

    if hasattr(axis2placement, 'Axis') and hasattr(axis2placement, 'RefDirection'):
        # 检查Axis和RefDirection是否为None
        if axis2placement.Axis is not None and axis2placement.RefDirection is not None:
            axis = np.array(axis2placement.Axis.DirectionRatios)
            ref_direction = np.array(axis2placement.RefDirection.DirectionRatios)
            # 构建旋转矩阵（简化处理，可能需要更复杂的计算）
            z = np.cross(axis, ref_direction)
            rotation_matrix = np.vstack((ref_direction, np.cross(z, ref_direction), z)).T
        else:
            rotation_matrix = np.eye(3)
    else:
        rotation_matrix = np.eye(3)

    translation_matrix = np.eye(4)
    translation_matrix[:3, 3] = location

    local_matrix = np.eye(4)
    local_matrix[:3, :3] = rotation_matrix

    combined_matrix = parent_matrix @ translation_matrix @ local_matrix
    return combined_matrix


def get_objects_bounding_boxes(ifc_file_path, ifc_type="IfcWall"):
    """Calculate complete bounding box data including all vertices"""
    try:
        ifc_file = ifcopenshell.open(ifc_file_path)
    except Exception as e:
        print(f"Failed to open IFC file: {e}")
        return {}

    settings = ifcopenshell.geom.settings()
    results = {}
    objects = ifc_file.by_type(ifc_type)
    if not objects:
        print(f"No objects of type {ifc_type} found in the IFC file.")
        return results

    for obj in objects:
        try:
            shape = ifcopenshell.geom.create_shape(settings, obj)
            verts = np.array(shape.geometry.verts).reshape(-1, 3)

            # 处理转换矩阵缺失的情况，默认使用单位矩阵
            transform_matrix = np.eye(4, dtype=np.float64)  # 初始化单位矩阵
            if hasattr(shape.geometry, 'transformation_matrix'):
                transform_matrix = np.array(shape.geometry.transformation_matrix, dtype=np.float64)
            elif hasattr(shape, 'transformation_matrix'):
                transform_matrix = np.array(shape.transformation_matrix, dtype=np.float64)
            else:
                print(f"Warning: No transformation matrix found for object {obj.GlobalId}, using identity matrix.")

            # 从IfcLocalPlacement获取全局转换矩阵
            placement = obj.ObjectPlacement
            global_placement_matrix = get_placement_matrix(placement)

            bbox_min, bbox_max = ifcopenshell.util.shape.get_bbox(verts)
            local_bbox_vertices = get_bbox_vertices(bbox_min, bbox_max)

            # 转换局部顶点到全局坐标（使用形状自带的转换矩阵和全局转换矩阵）
            global_vertices = []
            for vertex in local_bbox_vertices:
                local_global_vertex = local_to_global(transform_matrix, vertex)
                final_global_vertex = local_to_global(global_placement_matrix, local_global_vertex)
                global_vertices.append(final_global_vertex)

            global_vertices_np = np.array(global_vertices)
            global_bbox_min = np.min(global_vertices_np, axis=0)
            global_bbox_max = np.max(global_vertices_np, axis=0)

            results[obj.GlobalId] = {
                "type": obj.is_a(),
                "local_min": np.round(bbox_min, 3).tolist(),
                "local_max": np.round(bbox_max, 3).tolist(),
                "local_vertices": np.round(local_bbox_vertices, 3).tolist(),
                "min": np.round(global_bbox_min, 3).tolist(),
                "max": np.round(global_bbox_max, 3).tolist(),
                "vertices": np.round(global_vertices_np, 3).tolist(),
                "dimensions": np.round(global_bbox_max - global_bbox_min, 3).tolist()
            }

            print(f"Debug: Object {obj.GlobalId} - Transform Matrix:\n{transform_matrix}")
            print(f"Debug: Object {obj.GlobalId} - Global Placement Matrix:\n{global_placement_matrix}")
            print(f"Debug: Global Bounding Box: Min={global_bbox_min}, Max={global_bbox_max}\n")

        except Exception as e:
            print(f"Error processing object {obj.GlobalId}: {e}")
            results[obj.GlobalId] = {"error": str(e)}

    return results


# Example usage
if __name__ == "__main__":
    file_path = "/content/your_model.ifc"
    bboxes = get_objects_bounding_boxes(file_path, "IfcWall")

    if bboxes:
        for guid, data in bboxes.items():
            if "min" in data:
                print(f"\n{data['type']} {guid}:")
                print(f"Local Min: {data['local_min']}")
                print(f"Local Max: {data['local_max']}")
                print("Local All vertices:")
                for i, vertex in enumerate(data['local_vertices']):
                    print(f"  Local Vertex {i}: {vertex}")
                print(f"Global Min: {data['min']}")
                print(f"Global Max: {data['max']}")
                print(f"Dimensions (LxWxH): {data['dimensions']}")
                print("Global All vertices:")
                for i, vertex in enumerate(data['vertices']):
                    print(f"  Global Vertex {i}: {vertex}")
    else:
        print("No valid bounding box data was obtained.")


Debug: Object 072ZP5RUb3gBTy6zAYFpDy - Transform Matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Debug: Object 072ZP5RUb3gBTy6zAYFpDy - Global Placement Matrix:
[[ 1.          0.          0.         -2.94165764]
 [ 0.          1.          0.          1.03178204]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
Debug: Global Bounding Box: Min=[-2.94165764  0.88678204  0.        ], Max=[2.05834236 1.17678204 4.        ]


IfcWallStandardCase 072ZP5RUb3gBTy6zAYFpDy:
Local Min: [0.0, -0.145, 0.0]
Local Max: [5.0, 0.145, 4.0]
Local All vertices:
  Local Vertex 0: [0.0, -0.145, 0.0]
  Local Vertex 1: [5.0, -0.145, 0.0]
  Local Vertex 2: [5.0, 0.145, 0.0]
  Local Vertex 3: [0.0, 0.145, 0.0]
  Local Vertex 4: [0.0, -0.145, 4.0]
  Local Vertex 5: [5.0, -0.145, 4.0]
  Local Vertex 6: [5.0, 0.145, 4.0]
  Local Vertex 7: [0.0, 0.145, 4.0]
Global Min: [-2.942, 0.887, 0.0]
Global Max: [2.058, 1.177, 4.0]
Dimensions (LxWxH): [